## Stock Sentiment Analyzer

This is our team's sentiment analyzer. It aims to identify the sentiment behind groups of tweets from twitter in real time. 

We used this tutorial as a reference while creating our model: resource/tutorial used:
- https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

We also referenced other tutorials to find which one would work best for our usecase.
- https://realpython.com/sentiment-analysis-python/?fbclid=IwAR1sO-Zz3j9DQ9NdYiAciQIDTshzpqP_hxzPH-RjomOzlioXZeZjy_XDusE


In [1]:
import pandas as pd
import spacy as sp
from spacy.util import minibatch, compounding
import os
import random
import string
import json
from twython import Twython
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_colwidth', -1)

C:\Users\deepa\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [2]:
#read from stock kaggle dataset
stock_df = pd.read_csv('stock_data.csv')
stock_df.head()

,Text,Sentiment
0,"Kickers on my watchlist XIDE TIT SOQ PNK CPW BPZ AJ trade method 1 or method 2, see prev posts",1
1,user: AAP MOVIE. 55% return for the FEA/GEED indicator just 15 trades for the year. AWESOME.,1
2,user I'd be afraid to short AMZN - they are looking like a near-monopoly in eBooks and infrastructure-as-a-service,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


In [11]:
punctuations = string.punctuation


nlp = sp.load('en_core_web_sm')
stop_words = sp.lang.en.stop_words.STOP_WORDS


parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    #take out punctuation as well as per the library (it doesn't want punctuation)
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}


def clean_text(text):
    return text.strip().lower()

bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

X = stock_df['Text'] 
ylabels = stock_df['Sentiment'] 

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)


In [12]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001D0A4221198>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x000001D09E31BD90>)),
                ('classifier', LogisticRegression())])

In [13]:
from sklearn import metrics
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))
print('Logistic Regression F1:', metrics.f1_score(y_test, predicted))

Logistic Regression Accuracy: 0.7871116225546605
Logistic Regression Precision: 0.7991769547325103
Logistic Regression Recall: 0.8851412944393802
Logistic Regression F1: 0.8399653979238755


In [ ]:
print(type(X_test))
print(predicted[:3])

In [ ]:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate Twython object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

In [ ]:
# Search tweets
def search_twitter(ticker):
    # Create our query
    query = {'q': ticker,
        'result_type': 'recent',
        'count': 10,
        'lang': 'en',
        }
    tweets = []
    for status in python_tweets.search(**query)['statuses']:
        tweets.append(status['text'])
    return pd.Series(tweets)

In [ ]:
def find_predictions(tweets):
    predicted = pipe.predict(tweets)
    return predicted

In [ ]:
def display_split(predicted, ticker):
    neg = len(predicted[predicted == -1])
    pos = len(predicted[predicted == 1])

    labels = 'Positive', 'Negative'
    sizes = [pos, neg]

    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax1.axis('equal')  
    title = "Sentiment Split of " + ticker
    ax1.set_title(title)
    plt.show()

In [ ]:
# user interaction to find ticker information

ticker = '$TSLA' #input $ticker as a string
tweets = search_twitter(ticker)
print(tweets)
if (tweets.size == 0):
    print("There are no tweets about this company!")
else:
    predictions = find_predictions(tweets)
    display_split(predictions, ticker)